In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from shapely import wkt
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import f1_score, make_scorer

### **READ IN DATA**

In [3]:
train = pd.read_csv('data\Train.csv')

<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\T'
C:\Users\Aduragbemi\AppData\Local\Temp\ipykernel_19536\1100633172.py:1: SyntaxWarning: invalid escape sequence '\T'
  train = pd.read_csv('data\Train.csv')


In [4]:
train.category.value_counts()

category
Healthy     6506
Pests        485
Diseased     474
Stressed     423
Name: count, dtype: int64

In [5]:
test = pd.read_csv('data\Test.csv')

<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\T'
C:\Users\Aduragbemi\AppData\Local\Temp\ipykernel_19536\3210526593.py:1: SyntaxWarning: invalid escape sequence '\T'
  test = pd.read_csv('data\Test.csv')


In [6]:
test.head(3)

,FarmID,Crop,State,District,Sub-District,SDate,HDate,CropCoveredArea,CHeight,CNext,CLast,CTransp,IrriType,IrriSource,IrriCount,WaterCov,ExpYield,Season,geometry
0,85197,Paddy,Telangana,Medak,Nizampet,2023-11-12 0:00:00,2024-05-14 0:00:00,81,99,Lentil,Pea,Transplanting,Flood,Groundwater,5,81,19,Rabi,POLYGON ((78.54742059207287 18.062307326758276...
1,779677,Paddy,Telangana,Medak,Nizampet,2023-12-20 0:00:00,2024-05-28 0:00:00,91,63,Lentil,Pea,Transplanting,Flood,Canal,5,97,15,Rabi,"POLYGON ((78.54812514990353 18.06739449795583,..."
2,1331840,Paddy,Telangana,Medak,Nizampet,2023-11-29 0:00:00,2024-05-18 0:00:00,94,89,Pea,Lentil,Transplanting,Flood,Canal,5,88,15,Rabi,POLYGON ((78.56002792938853 18.079607077817514...


In [7]:
test.shape

(2718, 19)

In [8]:
test.columns

Index(['FarmID', 'Crop', 'State', 'District', 'Sub-District', 'SDate', 'HDate',
       'CropCoveredArea', 'CHeight', 'CNext', 'CLast', 'CTransp', 'IrriType',
       'IrriSource', 'IrriCount', 'WaterCov', 'ExpYield', 'Season',
       'geometry'],
      dtype='object')

In [9]:
train.shape

(7888, 20)

In [10]:
train.columns

Index(['FarmID', 'category', 'Crop', 'State', 'District', 'Sub-District',
       'SDate', 'HDate', 'CropCoveredArea', 'CHeight', 'CNext', 'CLast',
       'CTransp', 'IrriType', 'IrriSource', 'IrriCount', 'WaterCov',
       'ExpYield', 'Season', 'geometry'],
      dtype='object')

### **DATA WRANGLING**

In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7888 entries, 0 to 7887
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   FarmID           7888 non-null   int64 
 1   category         7888 non-null   object
 2   Crop             7888 non-null   object
 3   State            7888 non-null   object
 4   District         7888 non-null   object
 5   Sub-District     7888 non-null   object
 6   SDate            7888 non-null   object
 7   HDate            7888 non-null   object
 8   CropCoveredArea  7888 non-null   int64 
 9   CHeight          7888 non-null   int64 
 10  CNext            7888 non-null   object
 11  CLast            7888 non-null   object
 12  CTransp          7888 non-null   object
 13  IrriType         7888 non-null   object
 14  IrriSource       7888 non-null   object
 15  IrriCount        7888 non-null   int64 
 16  WaterCov         7888 non-null   int64 
 17  ExpYield         7888 non-null   

In [12]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2718 entries, 0 to 2717
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   FarmID           2718 non-null   int64 
 1   Crop             2718 non-null   object
 2   State            2718 non-null   object
 3   District         2718 non-null   object
 4   Sub-District     2718 non-null   object
 5   SDate            2718 non-null   object
 6   HDate            2718 non-null   object
 7   CropCoveredArea  2718 non-null   int64 
 8   CHeight          2718 non-null   int64 
 9   CNext            2718 non-null   object
 10  CLast            2718 non-null   object
 11  CTransp          2718 non-null   object
 12  IrriType         2718 non-null   object
 13  IrriSource       2718 non-null   object
 14  IrriCount        2718 non-null   int64 
 15  WaterCov         2718 non-null   int64 
 16  ExpYield         2718 non-null   int64 
 17  Season           2718 non-null   

In [13]:
train['geometry'][0]

'POLYGON ((78.18143246076087 17.978884603571068, 78.18149324901445 17.97898640558611, 78.18175397935514 17.978865561467536, 78.18166316316905 17.978728604799823, 78.18143246076087 17.978884603571068))'

### **SET _FarmID_ AS THE INDEX**

In [14]:
train.set_index('FarmID', inplace=True)

In [15]:
train.head(3)

,category,Crop,State,District,Sub-District,SDate,HDate,CropCoveredArea,CHeight,CNext,CLast,CTransp,IrriType,IrriSource,IrriCount,WaterCov,ExpYield,Season,geometry
FarmID,,,,,,,,,,,,,,,,,,,
1326576,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-11-25 0:00:00,2024-04-14 0:00:00,97,54,Pea,Lentil,Transplanting,Flood,Groundwater,4,87,17,Rabi,POLYGON ((78.18143246076087 17.978884603571068...
1326577,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-11-13 0:00:00,2024-04-26 0:00:00,82,58,Pea,Lentil,Transplanting,Flood,Canal,5,94,15,Rabi,POLYGON ((78.17545177547453 17.981074445477304...
1326578,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-12-19 0:00:00,2024-04-28 0:00:00,92,91,Pea,Lentil,Transplanting,Flood,Canal,3,99,20,Rabi,POLYGON ((78.16914224770792 17.976212849967755...


In [16]:
test.set_index('FarmID', inplace=True)

In [17]:
test.head(3)

,Crop,State,District,Sub-District,SDate,HDate,CropCoveredArea,CHeight,CNext,CLast,CTransp,IrriType,IrriSource,IrriCount,WaterCov,ExpYield,Season,geometry
FarmID,,,,,,,,,,,,,,,,,,
85197,Paddy,Telangana,Medak,Nizampet,2023-11-12 0:00:00,2024-05-14 0:00:00,81,99,Lentil,Pea,Transplanting,Flood,Groundwater,5,81,19,Rabi,POLYGON ((78.54742059207287 18.062307326758276...
779677,Paddy,Telangana,Medak,Nizampet,2023-12-20 0:00:00,2024-05-28 0:00:00,91,63,Lentil,Pea,Transplanting,Flood,Canal,5,97,15,Rabi,"POLYGON ((78.54812514990353 18.06739449795583,..."
1331840,Paddy,Telangana,Medak,Nizampet,2023-11-29 0:00:00,2024-05-18 0:00:00,94,89,Pea,Lentil,Transplanting,Flood,Canal,5,88,15,Rabi,POLYGON ((78.56002792938853 18.079607077817514...


### **CONVERT *SDate* AND *HDate* FEATURE TO DATETIME**

In [18]:
train['SDate'] = pd.to_datetime(train['SDate'])
train['HDate'] = pd.to_datetime(train['HDate']) 

print(train['SDate'].dtype)
print(train['HDate'].dtype)

datetime64[ns]
datetime64[ns]


In [19]:
test['SDate'] = pd.to_datetime(test['SDate'])
test['HDate'] = pd.to_datetime(test['HDate'])

print(test['SDate'].dtype)
print(test['HDate'].dtype)

datetime64[ns]
datetime64[ns]


### **EXTRACTING FEATURES FROM _geometry_**

In [20]:
train['geometry'] = train['geometry'].astype(str).apply(wkt.loads)  
train['Area'] = train['geometry'].apply(lambda x: x.area if x else None)
train['Perimeter'] = train['geometry'].apply(lambda x: x.length if x else None)   

train = train.drop(['geometry'], axis=1)

In [21]:
train.head(3)

,category,Crop,State,District,Sub-District,SDate,HDate,CropCoveredArea,CHeight,CNext,CLast,CTransp,IrriType,IrriSource,IrriCount,WaterCov,ExpYield,Season,Area,Perimeter
FarmID,,,,,,,,,,,,,,,,,,,,
1326576,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-11-25,2024-04-14,97,54,Pea,Lentil,Transplanting,Flood,Groundwater,4,87,17,Rabi,3.982611e-08,0.000849
1326577,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-11-13,2024-04-26,82,58,Pea,Lentil,Transplanting,Flood,Canal,5,94,15,Rabi,5.989901e-08,0.001016
1326578,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-12-19,2024-04-28,92,91,Pea,Lentil,Transplanting,Flood,Canal,3,99,20,Rabi,3.867260e-08,0.000798


In [22]:
test['geometry'] = test['geometry'].astype(str).apply(wkt.loads)  
test['Area'] = test['geometry'].apply(lambda x: x.area if x else None)
test['Perimeter'] = test['geometry'].apply(lambda x: x.length if x else None)   

test = test.drop(['geometry'], axis=1)

In [23]:
test.head(3)

,Crop,State,District,Sub-District,SDate,HDate,CropCoveredArea,CHeight,CNext,CLast,CTransp,IrriType,IrriSource,IrriCount,WaterCov,ExpYield,Season,Area,Perimeter
FarmID,,,,,,,,,,,,,,,,,,,
85197,Paddy,Telangana,Medak,Nizampet,2023-11-12,2024-05-14,81,99,Lentil,Pea,Transplanting,Flood,Groundwater,5,81,19,Rabi,1.011755e-07,0.001353
779677,Paddy,Telangana,Medak,Nizampet,2023-12-20,2024-05-28,91,63,Lentil,Pea,Transplanting,Flood,Canal,5,97,15,Rabi,1.679801e-07,0.002157
1331840,Paddy,Telangana,Medak,Nizampet,2023-11-29,2024-05-18,94,89,Pea,Lentil,Transplanting,Flood,Canal,5,88,15,Rabi,7.013044e-08,0.001091


### **EXTRACT *year*, *month* AND *day* FROM *SDate* AND *HDate***

In [24]:
train['SYear'] = train['SDate'].dt.year
train['SMonth'] = train['SDate'].dt.month
train['SDay'] = train['SDate'].dt.day

train = train.drop(['SDate'], axis=1)

train['HYear'] = train['HDate'].dt.year
train['HMonth'] = train['HDate'].dt.month
train['HDay'] = train['HDate'].dt.day

train = train.drop(['HDate'], axis=1)

train.head(3)

,category,Crop,State,District,Sub-District,CropCoveredArea,CHeight,CNext,CLast,CTransp,...,ExpYield,Season,Area,Perimeter,SYear,SMonth,SDay,HYear,HMonth,HDay
FarmID,,,,,,,,,,,,,,,,,,,,,
1326576,Healthy,Paddy,Telangana,Medak,Kulcharam,97,54,Pea,Lentil,Transplanting,...,17,Rabi,3.982611e-08,0.000849,2023,11,25,2024,4,14
1326577,Healthy,Paddy,Telangana,Medak,Kulcharam,82,58,Pea,Lentil,Transplanting,...,15,Rabi,5.989901e-08,0.001016,2023,11,13,2024,4,26
1326578,Healthy,Paddy,Telangana,Medak,Kulcharam,92,91,Pea,Lentil,Transplanting,...,20,Rabi,3.867260e-08,0.000798,2023,12,19,2024,4,28


In [25]:
test['SYear'] = test['SDate'].dt.year
test['SMonth'] = test['SDate'].dt.month
test['SDay'] = test['SDate'].dt.day

test = test.drop(['SDate'], axis=1)

test['HYear'] = test['HDate'].dt.year
test['HMonth'] = test['HDate'].dt.month
test['HDay'] = test['HDate'].dt.day

test = test.drop(['HDate'], axis=1)

test.head(3)

,Crop,State,District,Sub-District,CropCoveredArea,CHeight,CNext,CLast,CTransp,IrriType,...,ExpYield,Season,Area,Perimeter,SYear,SMonth,SDay,HYear,HMonth,HDay
FarmID,,,,,,,,,,,,,,,,,,,,,
85197,Paddy,Telangana,Medak,Nizampet,81,99,Lentil,Pea,Transplanting,Flood,...,19,Rabi,1.011755e-07,0.001353,2023,11,12,2024,5,14
779677,Paddy,Telangana,Medak,Nizampet,91,63,Lentil,Pea,Transplanting,Flood,...,15,Rabi,1.679801e-07,0.002157,2023,12,20,2024,5,28
1331840,Paddy,Telangana,Medak,Nizampet,94,89,Pea,Lentil,Transplanting,Flood,...,15,Rabi,7.013044e-08,0.001091,2023,11,29,2024,5,18


### **DROP *State* FEATURE**

In [26]:
train = train.drop(['State'], axis=1)
test = test.drop(['State'], axis=1)

In [27]:
train.describe()

,CropCoveredArea,CHeight,IrriCount,WaterCov,ExpYield,Area,Perimeter,SYear,SMonth,SDay,HYear,HMonth,HDay
count,7888.000000,7888.000000,7888.000000,7888.000000,7888.000000,7.888000e+03,7888.000000,7888.000000,7888.000000,7888.000000,7888.000000,7888.000000,7888.000000
mean,77.419244,87.448783,3.538159,57.154285,15.694473,3.614335e-07,0.002310,2023.015340,8.439528,16.619549,2023.781694,5.626395,15.225279
std,14.479677,50.308776,1.405152,22.860627,5.646621,3.968992e-07,0.001213,0.122908,3.299687,7.479800,0.413123,3.557494,9.201316
min,40.000000,20.000000,1.000000,20.000000,5.000000,4.582125e-09,0.000274,2023.000000,1.000000,1.000000,2023.000000,1.000000,1.000000
25%,66.000000,54.000000,2.000000,41.000000,11.000000,1.065698e-07,0.001395,2023.000000,6.000000,11.000000,2024.000000,3.000000,5.000000
50%,79.000000,76.000000,4.000000,51.000000,15.000000,2.369511e-07,0.002100,2023.000000,10.000000,16.000000,2024.000000,4.000000,16.000000
75%,89.000000,100.000000,5.000000,82.000000,20.000000,4.722745e-07,0.003003,2023.000000,11.000000,23.000000,2024.000000,9.000000,23.000000
max,100.000000,250.000000,6.000000,100.000000,30.000000,5.949748e-06,0.010898,2024.000000,12.000000,31.000000,2024.000000,12.000000,31.000000


In [28]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7888 entries, 1326576 to 1330504
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   category         7888 non-null   object 
 1   Crop             7888 non-null   object 
 2   District         7888 non-null   object 
 3   Sub-District     7888 non-null   object 
 4   CropCoveredArea  7888 non-null   int64  
 5   CHeight          7888 non-null   int64  
 6   CNext            7888 non-null   object 
 7   CLast            7888 non-null   object 
 8   CTransp          7888 non-null   object 
 9   IrriType         7888 non-null   object 
 10  IrriSource       7888 non-null   object 
 11  IrriCount        7888 non-null   int64  
 12  WaterCov         7888 non-null   int64  
 13  ExpYield         7888 non-null   int64  
 14  Season           7888 non-null   object 
 15  Area             7888 non-null   float64
 16  Perimeter        7888 non-null   float64
 17  SYear     

### **CHANGE DATA TYPE OF NON-NUMERIC FEATURE TO _Category_**

In [29]:
train_cat_cols = train.select_dtypes(include='object').columns

for col in train_cat_cols:
    train[col] = train[col].astype('category')

train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7888 entries, 1326576 to 1330504
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   category         7888 non-null   category
 1   Crop             7888 non-null   category
 2   District         7888 non-null   category
 3   Sub-District     7888 non-null   category
 4   CropCoveredArea  7888 non-null   int64   
 5   CHeight          7888 non-null   int64   
 6   CNext            7888 non-null   category
 7   CLast            7888 non-null   category
 8   CTransp          7888 non-null   category
 9   IrriType         7888 non-null   category
 10  IrriSource       7888 non-null   category
 11  IrriCount        7888 non-null   int64   
 12  WaterCov         7888 non-null   int64   
 13  ExpYield         7888 non-null   int64   
 14  Season           7888 non-null   category
 15  Area             7888 non-null   float64 
 16  Perimeter        7888 non-null   float

In [30]:
test_cat_cols = test.select_dtypes(include='object').columns

for col in test_cat_cols:
    test[col] = test[col].astype('category')

test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2718 entries, 85197 to 1326552
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   Crop             2718 non-null   category
 1   District         2718 non-null   category
 2   Sub-District     2718 non-null   category
 3   CropCoveredArea  2718 non-null   int64   
 4   CHeight          2718 non-null   int64   
 5   CNext            2718 non-null   category
 6   CLast            2718 non-null   category
 7   CTransp          2718 non-null   category
 8   IrriType         2718 non-null   category
 9   IrriSource       2718 non-null   category
 10  IrriCount        2718 non-null   int64   
 11  WaterCov         2718 non-null   int64   
 12  ExpYield         2718 non-null   int64   
 13  Season           2718 non-null   category
 14  Area             2718 non-null   float64 
 15  Perimeter        2718 non-null   float64 
 16  SYear            2718 non-null   int32  

### **ENCODE _Categorical_ FEATURES**

In [31]:
for col in train_cat_cols:
    train[col] = train[col].cat.codes

train.head(3)

,category,Crop,District,Sub-District,CropCoveredArea,CHeight,CNext,CLast,CTransp,IrriType,...,ExpYield,Season,Area,Perimeter,SYear,SMonth,SDay,HYear,HMonth,HDay
FarmID,,,,,,,,,,,,,,,,,,,,,
1326576,1,5,5,61,97,54,4,0,3,1,...,17,1,3.982611e-08,0.000849,2023,11,25,2024,4,14
1326577,1,5,5,61,82,58,4,0,3,1,...,15,1,5.989901e-08,0.001016,2023,11,13,2024,4,26
1326578,1,5,5,61,92,91,4,0,3,1,...,20,1,3.867260e-08,0.000798,2023,12,19,2024,4,28


In [32]:
for col in test_cat_cols:
    test[col] = test[col].cat.codes

test.head(3)

,Crop,District,Sub-District,CropCoveredArea,CHeight,CNext,CLast,CTransp,IrriType,IrriSource,...,ExpYield,Season,Area,Perimeter,SYear,SMonth,SDay,HYear,HMonth,HDay
FarmID,,,,,,,,,,,,,,,,,,,,,
85197,5,5,25,81,99,0,4,3,1,1,...,19,1,1.011755e-07,0.001353,2023,11,12,2024,5,14
779677,5,5,25,91,63,0,4,3,1,0,...,15,1,1.679801e-07,0.002157,2023,12,20,2024,5,28
1331840,5,5,25,94,89,4,0,3,1,0,...,15,1,7.013044e-08,0.001091,2023,11,29,2024,5,18


### **MODEL BUILDING**

In [33]:
X = train.drop(['category'], axis=1)
y = train['category']

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [35]:
# Oversampling the minority class
sm = SMOTE(random_state=42)
X_train, y_train = sm.fit_resample(X_train, y_train)

### **SCALING**

In [36]:
from sklearn.preprocessing import StandardScaler    

In [37]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [38]:
clf = RandomForestClassifier(n_estimators=500, random_state=42)
clf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=500, random_state=42)

In [39]:
y_pred = clf.predict(X_test)

In [40]:
# F1 score
f1_score(y_test, y_pred, average='weighted')

0.7400350583093692

### **PREDICTION ON TEST SET**

In [41]:
test_pred = clf.predict(test)
test_pred

c:\Users\Aduragbemi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


array([1, 1, 1, ..., 1, 1, 1], dtype=int8)

In [42]:
submission = pd.DataFrame(
    {
        'ID': test.index,
        'Target': test_pred
    }
)

In [43]:
submission.head(5)

,ID,Target
0,85197,1
1,779677,1
2,1331840,1
3,483053,1
4,542945,1


In [44]:
category_map = {0: 'Diseased', 1: 'Healthy', 2: 'Pests', 3: 'Stressed'}

submission['Target'] = submission['Target'].map(category_map)
submission.head(3)

,ID,Target
0,85197,Healthy
1,779677,Healthy
2,1331840,Healthy


In [45]:
submission.to_csv('submission.csv', index=False)

In [46]:
param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

# Define the scorer
scorer = make_scorer(f1_score, average='weighted')

In [ ]:
random_search = RandomizedSearchCV(estimator=clf, 
                                   param_distributions=param_dist, 
                                   scoring=scorer, 
                                   n_iter=10, 
                                   cv=5, 
                                   random_state=42)
random_search.fit(X_train, y_train)

random_search.best_params_

In [ ]:
best_model = random_search.best_estimator_

# Evaluate on the test set
y_test_pred = best_model.predict(X_test)
f1_score(y_test, y_test_pred, average='weighted')

0.7385465039735567

### **BUILDING WITH XGBOOST**

In [49]:
import xgboost as xgb

In [ ]:
model = xgb.XGBClassifier(objective='multi:softmax', num_class=4, enable_categorical=True, random_state=42)

In [ ]:
data = pd.read_csv('Data.csv')
data.head()

,FarmID,category,Crop,State,District,Sub-District,SDate,HDate,CropCoveredArea,CHeight,...,CTransp,IrriType,IrriSource,IrriCount,WaterCov,ExpYield,Season,geometry,dataset,tif_path
0,1326576,Healthy,Paddy,Telangana,Medak,Kulcharam,25-11-2023,14-04-2024,97,54,...,Transplanting,Flood,Groundwater,4,87,17,Rabi,POLYGON ((78.18079255482755 17.978971746424413...,train,/kaggle/input/sentineltimeseriesdata/SentinelT...
1,1326577,Healthy,Paddy,Telangana,Medak,Kulcharam,13-11-2023,26-04-2024,82,58,...,Transplanting,Flood,Canal,5,94,15,Rabi,POLYGON ((78.17483419891283 17.981508840879556...,train,/kaggle/input/sentineltimeseriesdata/SentinelT...
2,1326578,Healthy,Paddy,Telangana,Medak,Kulcharam,19-12-2023,28-04-2024,92,91,...,Transplanting,Flood,Canal,3,99,20,Rabi,POLYGON ((78.16888476438905 17.976727511659835...,train,/kaggle/input/sentineltimeseriesdata/SentinelT...
3,1331836,Diseased,Paddy,Telangana,Medak,Kulcharam,11-02-2023,04-11-2024,91,52,...,Transplanting,Flood,Canal,5,92,16,Rabi,POLYGON ((78.16821795882959 17.974690113471414...,train,/kaggle/input/sentineltimeseriesdata/SentinelT...
4,1326579,Diseased,Paddy,Telangana,Medak,Kulcharam,12-12-2023,19-05-2024,94,55,...,Transplanting,Flood,Canal,5,97,20,Rabi,POLYGON ((78.17244459940454 17.969620948965225...,train,/kaggle/input/sentineltimeseriesdata/SentinelT...
